In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_ml_control
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
import warnings

warnings.simplefilter("ignore", UserWarning)

In [ ]:
%autoreload

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from do_mpc.simulator import Simulator
from numpy.typing import NDArray

from training_ml_control.environments import (
    create_cart_environment,
    create_inverted_pendulum_environment,
    simulate_environment,
)
from training_ml_control.plots import (
    animate_cart_simulation,
    animate_inverted_pendulum_simulation,
    animate_full_inverted_pendulum_simulation,
)
from training_ml_control.models import (
    build_cart_model,
    build_inverted_pendulum_linear_model,
    build_inverted_pendulum_nonlinear_model,
)

from training_ml_control.plots import (
    plot_cart_results,
    plot_inverted_pendulum_results,
)
from training_ml_control.nb_utils import (
    show_video,
)

sns.set_theme()
plt.rcParams["figure.figsize"] = [9, 5]

```{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
```

# Systems

## Cart (Double Integrator)

The double integrator is a canonical example of a second-order control system. It models the dynamics of a simple mass in one-dimensional space under the effect of a time-varying force input. 

In [ ]:
cart_env = create_cart_environment(goal_position=9)
results = simulate_environment(cart_env)
show_video(results.frames, fps=1 / cart_env.dt)

In [ ]:
T = np.arange(len(results.observations)) * cart_env.dt
plot_cart_results(
    T=T,
    observations=results.observations,
    actions=results.actions,
    reference=cart_env.goal_position,
)

For the simulation we will use a modified version of the [Mountain Car Continuous](https://gymnasium.farama.org/environments/classic_control/mountain_car_continuous/) environment from [gymnasium](https://gymnasium.farama.org/).

It has the following possible action and observations:

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>Num</p></th>
<th class="head"><p>Action</p></th>
<th class="head"><p>Control Min</p></th>
<th class="head"><p>Control Max</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>0</p></td>
<td><p>Force applied on the cart</p></td>
<td><p>-10</p></td>
<td><p>10</p></td>
</tr>
</tbody>
</table>

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>Num</p></th>
<th class="head"><p>Observation</p></th>
<th class="head"><p>Min</p></th>
<th class="head"><p>Max</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>0</p></td>
<td><p>position of the cart along the linear surface</p></td>
<td><p>-3.0</p></td>
<td><p>3.0</p></td>
</tr>
<tr class="row-even"><td><p>1</p></td>
<td><p>linear velocity of the cart</p></td>
<td><p>-Inf</p></td>
<td><p>Inf</p></td>
</tr>
</tr>
</tbody>
</table>

### Model

The differential equations which represent a one-dimensional double integrator are:

$$
\begin{array}{ll}
\ddot {q}(t) &= u(t)\\
y(t) &= q(t)
\end{array}
$$

where $\displaystyle q(t),u(t)\in \mathbb {R}$.

It is clear that the control input $u$ is the second derivative of the output $q$.

Defining the state-vector vector as $X = \begin{bmatrix}x_1\\{x_2}\\\end{bmatrix} = \begin{bmatrix}q\\{\dot {q}}\\\end{bmatrix}$, gives us the following state-space representation:

$$
\dot{X} = \begin{bmatrix}{\dot{x}_1}\\{\dot {x}_2}\\\end{bmatrix} = \begin{bmatrix}{x_2}\\{u}\\\end{bmatrix}
$$

This leads to a linear state-space model with matrices:

$$
A = \begin{bmatrix}
0 & 1 \\
0 & 0\\
\end{bmatrix};
B = \begin{bmatrix}
0 \\ 1
\end{bmatrix};
C = \begin{bmatrix}
1 & 0 \\
\end{bmatrix};
D = \begin{bmatrix}
0
\end{bmatrix}
$$

In [ ]:
cart_model = build_cart_model(cart_env)

### Simulator

In [ ]:
cart_simulator = Simulator(cart_model)
cart_simulator.set_param(t_step=cart_env.dt)
cart_simulator.setup()

We then use the simulator to simulate the model starting from the same initial state as the environment and using the same inputs.

In [ ]:
cart_simulator.reset_history()
x0 = results.observations[0]
cart_simulator.x0 = x0
for i in range(len(results.observations) - 1):
    u = results.actions[[i]]
    x0 = cart_simulator.make_step(u)

animate_cart_simulation(cart_simulator.data)

## Inverted Pendulum

```{figure} _static/images/10_inverted_pendulum_photo.png
---
name: inverted-pendulum
---
Balancing cart, a simple robotics system circa 1976 - [Wikipedia](https://en.wikipedia.org/wiki/Inverted_pendulum).
```

An inverted pendulum is a pendulum that has its center of mass above its pivot point. It is unstable and without additional help will fall over.

The inverted pendulum is a classic problem in dynamics and control theory and is used as a benchmark for testing control strategies. It is often implemented with the pivot point mounted on a cart that can move horizontally under control of an electronic servo system as shown in the image.

In [ ]:
%%html
<iframe width="800" height="600" src="https://www.youtube-nocookie.com/embed/AuAZ5zOP0yQ?si=1Lnyg2ghX6BJEEVX&amp;start=55" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

For the simulation we will use a modified version of the [CartPole](https://gymnasium.farama.org/environments/classic_control/cart_pole/) environment from [gymnasium](https://gymnasium.farama.org/).

It has the following possible action and observations:

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>Num</p></th>
<th class="head"><p>Action</p></th>
<th class="head"><p>Control Min</p></th>
<th class="head"><p>Control Max</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>0</p></td>
<td><p>Force applied on the cart</p></td>
<td><p>-30</p></td>
<td><p>30</p></td>
</tr>
</tbody>
</table>

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>Num</p></th>
<th class="head"><p>Observation</p></th>
<th class="head"><p>Min</p></th>
<th class="head"><p>Max</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>0</p></td>
<td><p>position of the cart along the linear surface</p></td>
<td><p>-3.0</p></td>
<td><p>3.0</p></td>
</tr>
<tr class="row-even"><td><p>1</p></td>
<td><p>linear velocity of the cart</p></td>
<td><p>-Inf</p></td>
<td><p>Inf</p></td>
</tr>
</tr>
<tr class="row-odd"><td><p>2</p></td>
<td><p>vertical angle of the pole on the cart</p></td>
<td><p>-24</p></td>
<td><p>24</p></td>
<tr class="row-odd"><td><p>3</p></td>
<td><p>angular velocity of the pole on the cart</p></td>
<td><p>-Inf</p></td>
<td><p>Inf</p></td>
</tr>
</tbody>
</table>

In [ ]:
inverted_pendulum_env = create_inverted_pendulum_environment(
    max_steps=100, theta_threshold=np.inf
)
results = simulate_environment(inverted_pendulum_env)
show_video(results.frames, fps=1 / inverted_pendulum_env.dt)

In [ ]:
T = np.arange(len(results.observations)) * inverted_pendulum_env.dt
plot_inverted_pendulum_results(
    T=T, observations=results.observations, actions=results.actions, reference=np.inf
)

### Dynamic Model

:::{figure} _static/images/20_inverted_pendulum_diagram.svg
:width: 40%

Inverted pendulum model diagram. *Taken from [this document](https://coneural.org/florian/papers/05_cart_pole.pdf)*.
:::
    
- $x$: distance along the horizontal axis from some reference point.
- $\theta$: angle of the pendulum.
- $m_c$: mass of the cart.
- $m_p$: mass of the pole.
- $l$: length of the pole.
- $F$: force applied on the cart.

Application of Newtonian physics to this system leads to the following model[^1][^2]:

[^1]: for the sake of simplicity we will ignore friction.
[^2]: for a detailed derivation of these equations refer to [this document](https://sharpneat.sourceforge.io/research/cart-pole/cart-pole-equations.html).

$$
\ddot{x} = \frac{1}{m_p\cos^2(\theta) - \frac{4}{3}(m_c + m_p)} \left[
m_p g \sin(\theta) \cos(\theta) - \frac{4}{3}(F + m_p \frac{l}{2} \dot{\theta}^2 \sin(\theta))
\right]\\
\ddot{\theta} = \frac{1}{\frac{4}{3}(m_c + m_p)l - m_p l \cos^2(\theta)} \left[
(m_c + m_p)g\sin(\theta) - \cos(\theta) (F + m_p l \dot{\theta}^2 \sin(\theta))
\right]
$$

We can convert this to a state-space representation with input $\mathbf{u} = F$ and state$\mathbf{X}$:

$$
\mathbf{X} = \begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{bmatrix}
= \begin{bmatrix}
x \\ \dot{x} \\ \theta \\ \dot{\theta}
\end{bmatrix}
$$

$$
\dot{\mathbf{X}} = \begin{bmatrix}
\dot{x}_1 \\ \dot{x}_2 \\ \dot{x}_3 \\ \dot{x}_4
\end{bmatrix}
= \begin{bmatrix}
x_2 \\
\frac{1}{m_p\cos^2(x_3) - \frac{4}{3}(m_c + m_p)} \left[
m_p g \sin(x_3) \cos(x_3) - \frac{4}{3}(F + m_p l x_4^2 \sin(x_3))
\right]\\ \\
x_4 \\
\frac{1}{\frac{4}{3}(m_c + m_p)l - m_p l \cos^2(x_3)} \left[
(m_c + m_p)g\sin(x_3) - \cos(x_3) (\mathbf{u} + m_p l x_4^2 \sin(x_3))
\right] \\
\end{bmatrix}
$$

For simple cases (e.g. linear controller design), we're only interested in controlling the pendulum's angle, so we can ignore the first 2 states
and redefine the state vector as.

$$
\mathbf{X} = \begin{bmatrix}
x_1 \\ x_2
\end{bmatrix}
= \begin{bmatrix}
\theta \\ \dot{\theta}
\end{bmatrix}
$$

Which gives us:

$$
\dot{\mathbf{X}} = \begin{bmatrix}
\dot{x}_1 \\ \dot{x}_2
\end{bmatrix} =
\begin{bmatrix}
x_2 \\
\frac{1}{\frac{4}{3}(m_c + m_p)l - m_p l\cos^2(x_1)} \left[
(m_c + m_p)g\sin(x_1) - \cos(x_1) (\mathbf{u} + m_p l x_2^2 \sin(x_1))\right] \\
\end{bmatrix}
$$

These equations are nonlinear. However, for small departures of $x_1$ (i.e. $\theta$) from the
vertical position we can linearize about $x_1 = 0$, $x_2 = 0$. This is called the [small-angle approximation](https://en.wikipedia.org/wiki/Small-angle_approximation).

Applying the approximation gives us:

$$
\dot{\mathbf{X}} = \begin{bmatrix}
\dot{x}_1 \\ \dot{x}_2
\end{bmatrix} =
\begin{bmatrix}
x_2 \\
\frac{1}{m_c l} \left[
- \mathbf{u} + (m_c + m_p) g x_1
\right] \\
\end{bmatrix} =
\begin{bmatrix}
0 & 1\\
\frac{(m_c + m_p)g}{m_c l} & 0\\
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2
\end{bmatrix}
+
\begin{bmatrix}
0 \\ -\frac{1}{m_c l}
\end{bmatrix}
\begin{bmatrix}
\mathbf{u}\\
\end{bmatrix}
$$

This leads to a linear state-space model with matrices:

$$
A = \begin{bmatrix}
0 & 1 \\
\frac{(m_c + m_p)g}{m_c l} & 0\\
\end{bmatrix};
B = \begin{bmatrix}
0 \\ -\frac{1}{m_c l}
\end{bmatrix};
C = \begin{bmatrix}
1 & 0 \\
\end{bmatrix};
D = \begin{bmatrix}
0
\end{bmatrix}
$$

### Linear Model

In [ ]:
inverted_pendulum_linear_model = build_inverted_pendulum_linear_model(
    inverted_pendulum_env
)

### Simulator

In [ ]:
inverted_pendulum_linear_simulator = Simulator(inverted_pendulum_linear_model)
inverted_pendulum_linear_simulator.set_param(t_step=inverted_pendulum_env.dt)
inverted_pendulum_linear_simulator.setup()

We then use the simulator to simulate the the model starting from the same initial state as the environment and using the same inputs.

In [ ]:
x0 = results.observations[0, 2:]

inverted_pendulum_linear_simulator.reset_history()
inverted_pendulum_linear_simulator.x0 = x0

for i in range(len(results.observations) - 1):
    u = results.actions[[i]]
    x0 = inverted_pendulum_linear_simulator.make_step(u)

animate_inverted_pendulum_simulation(inverted_pendulum_linear_simulator.data)

We notice that the simulation quickly diverges as the states moves away from the origin. The model should still be good for the purpose of controlling the system to stay near the origin.

### Full Non-Linear Model

In [ ]:
inverted_pendulum_nonlinear_model = build_inverted_pendulum_nonlinear_model(
    inverted_pendulum_env
)

### Simulator

In [ ]:
inverted_pendulum_nonlinear_simulator = Simulator(inverted_pendulum_nonlinear_model)
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    "integration_tool": "idas",
    "abstol": 1e-8,
    "reltol": 1e-8,
    "t_step": inverted_pendulum_env.dt,
}
inverted_pendulum_nonlinear_simulator.set_param(**params_simulator)
inverted_pendulum_nonlinear_simulator.setup()

We then use the simulator to simulate the the model starting from the same initial state as the environment and using the same inputs.

In [ ]:
x0 = results.observations[0]

inverted_pendulum_nonlinear_simulator.reset_history()
inverted_pendulum_nonlinear_simulator.x0 = x0

for i in range(len(results.observations) - 1):
    u = results.actions[[i]]
    x0 = inverted_pendulum_nonlinear_simulator.make_step(u)

animate_full_inverted_pendulum_simulation(inverted_pendulum_nonlinear_simulator.data)